In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
df = pd.read_csv('C:/Users/levin/Desktop/loan/loan.csv')

C:\Users\levin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
loans = df[(df['loan_status']=='Fully Paid')|(df['loan_status']=='Charged Off')|(df['loan_status']=='Default')]

In [3]:
def is_fully_paid(row):
    if row['loan_status']  == 'Fully Paid':
        return 1
    else:
        return 0
loans['new_status'] = loans.apply(lambda x: is_fully_paid(x), axis=1)
loans['new_status'].value_counts()

C:\Users\levin\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1    207723
0     46467
Name: new_status, dtype: int64

Feature selection:
features=['loan_amnt','funded_amnt','term','int_rate','installment','sub_grade','home_ownership','annual_inc','verification_status',         'delinq_2yrs','inq_last_6mths','open_acc','pub_rec','revol_bal','total_acc','out_prncp','total_pymnt','total_rec_prncp',
   'total_rec_int','total_rec_late_fee','acc_now_delinq']
categorical_features=['term','sub_grade','home_ownership','verification_status']

## GET DUMMY VARIABLES for Categorical data
---

In [10]:
variables = ['loan_amnt','funded_amnt','term','int_rate','installment','sub_grade','home_ownership','annual_inc','verification_status',
         'delinq_2yrs','inq_last_6mths','open_acc','pub_rec','revol_bal','total_acc','out_prncp','total_pymnt','total_rec_prncp',
         'total_rec_int','total_rec_late_fee','acc_now_delinq']
loans_data = loans[variables]
loans_target = loans['new_status']

In [11]:
def convert_categorical(df,col):
    dummy = pd.get_dummies(df[col])
    new_df = pd.concat([df,dummy],axis=1)
    new_df.drop(col,axis=1,inplace=True)
    return new_df

for v in ['term','sub_grade','home_ownership','verification_status']:
    loans_data = convert_categorical(loans_data,v)

In [12]:
loans_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254190 entries, 0 to 887371
Data columns (total 63 columns):
loan_amnt             254190 non-null float64
funded_amnt           254190 non-null float64
int_rate              254190 non-null float64
installment           254190 non-null float64
annual_inc            254190 non-null float64
delinq_2yrs           254190 non-null float64
inq_last_6mths        254190 non-null float64
open_acc              254190 non-null float64
pub_rec               254190 non-null float64
revol_bal             254190 non-null float64
total_acc             254190 non-null float64
out_prncp             254190 non-null float64
total_pymnt           254190 non-null float64
total_rec_prncp       254190 non-null float64
total_rec_int         254190 non-null float64
total_rec_late_fee    254190 non-null float64
acc_now_delinq        254190 non-null float64
 36 months            254190 non-null float64
 60 months            254190 non-null float64
A1             

## Split data into train set and test set
---

In [13]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(loans_data,loans_target,test_size=.2,random_state=1)

In [14]:
print(len(x_train),len(y_train))

203352 203352


In [ ]:
'''
from sklearn import preprocessing
variables=['loan_amnt','funded_amnt','term','int_rate','installment','sub_grade','home_ownership','annual_inc','verification_status',
         'delinq_2yrs','inq_last_6mths','open_acc','pub_rec','revol_bal','total_acc','out_prncp','total_pymnt','total_rec_prncp',
         'total_rec_int','total_rec_late_fee','acc_now_delinq','new_status']
features=['loan_amnt','funded_amnt','term','int_rate','installment','sub_grade','home_ownership','annual_inc','verification_status',
         'delinq_2yrs','inq_last_6mths','open_acc','pub_rec','revol_bal','total_acc','out_prncp','total_pymnt','total_rec_prncp',
         'total_rec_int','total_rec_late_fee','acc_now_delinq']
cate_features=['term','sub_grade','home_ownership','verification_status']

for f in cate_features:
    le = preprocessing.LabelEncoder()
    loans[f] = le.fit_transform(loans[f])

from sklearn.cross_validation import train_test_split
train,test =train_test_split(loans[variables] , test_size = .2)
train.shape
'''

In [15]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 150, min_samples_leaf = 50 , random_state = 1)
rfc.fit(x_train,y_train)
pred = rfc.predict(x_test)

train_pred = rfc.predict(x_train)
train_acc = sum(train_pred == y_train)/x_train.shape[0]
test_acc = sum(pred == y_test)/x_test.shape[0]
print('Random Forest Train accuracy is:{0}'.format(train_acc))
print('Random Forest Test accuracy is:{0}'.format(test_acc))

Random Forest Train accuracy is:0.9883354970691215
Random Forest Test accuracy is:0.9873323104764152


The test accuracy using Random Forest classifier is around **98.7%**  
The train accuracy is around **98.8%**  
No obvious overfitting here

## Normalize numeric data before fit in SVM
---

In [17]:
numeric_var = ['loan_amnt','funded_amnt','int_rate','installment','annual_inc','delinq_2yrs','inq_last_6mths','open_acc','pub_rec',
               'revol_bal','total_acc','out_prncp','total_pymnt','total_rec_prncp',
               'total_rec_int','total_rec_late_fee','acc_now_delinq']


In [18]:
def normalize(col):
    mean_ = col.mean()
    std = col.std()
    new_col = (col-mean_)/std
    return new_col

loans_data_norm = loans_data

for var in numeric_var:
    loans_data_norm[var] = normalize(loans_data_norm[var])

In [19]:
loans_data_norm.describe()

,loan_amnt,funded_amnt,int_rate,installment,annual_inc,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,G5,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,Not Verified,Source Verified,Verified
count,2.541900e+05,2.541900e+05,2.541900e+05,2.541900e+05,2.541900e+05,2.541900e+05,2.541900e+05,2.541900e+05,2.541900e+05,2.541900e+05,...,254190.000000,254190.000000,254190.000000,254190.000000,254190.000000,254190.000000,254190.000000,254190.000000,254190.000000,254190.000000
mean,-3.859041e-14,-1.646997e-14,-6.960899e-12,-1.770681e-14,1.493934e-15,-8.127882e-14,2.517970e-14,3.665575e-14,-3.089554e-13,-6.601097e-16,...,0.000751,0.000004,0.493104,0.000169,0.000555,0.086923,0.419245,0.339675,0.292982,0.367343
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.027402,0.001983,0.499953,0.013005,0.023546,0.281723,0.493437,0.473600,0.455131,0.482082
min,-1.607029e+00,-1.606482e+00,-1.920037e+00,-1.643783e+00,-1.182444e+00,-3.374685e-01,-7.989103e-01,-2.236600e+00,-3.291058e-01,-8.092215e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.771167e-01,-7.804456e-01,-6.893627e-01,-7.297019e-01,-4.679817e-01,-3.374685e-01,-7.989103e-01,-8.060560e-01,-3.291058e-01,-4.964738e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-1.931042e-01,-1.886582e-01,-5.586032e-02,-2.165801e-01,-1.787945e-01,-3.374685e-01,1.382191e-01,-1.929655e-01,-3.291058e-01,-2.257601e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.753334e-01,5.664872e-01,6.298662e-01,5.278487e-01,2.464809e-01,-3.374685e-01,1.382191e-01,6.244885e-01,-3.291058e-01,2.079552e-01,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,2.634746e+00,2.646990e+00,3.454515e+00,4.108799e+00,1.468743e+02,3.872558e+01,6.698125e+00,1.329503e+01,3.400186e+01,9.236515e+01,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Split data into train set and test set for SVM
---------

In [21]:
x_train_norm, x_test_norm, y_train, y_test = train_test_split(loans_data_norm,loans_target,test_size=.2,random_state=1)

In [23]:
from sklearn import svm
lin_clf = svm.LinearSVC(random_state=1)
lin_clf.fit(x_train_norm,y_train)

linsvc_test_pred = lin_clf.predict(x_test_norm)
linsvc_train_pred = lin_clf.predict(x_train_norm)
train_acc_linsvc = sum(linsvc_train_pred == y_train)/x_train_norm.shape[0]
test_acc_linsvc = sum(linsvc_test_pred == y_test)/x_test_norm.shape[0]
print('Linear SVC train accuracy:{0}'.format(train_acc_linsvc))
print('Linear SVC test accuracy:{0}'.format(test_acc_linsvc))

Linear SVC train accuracy:0.9885764585546245
Linear SVC test accuracy:0.9877650576340533


The Linear kernel SVM classifer is doing a little better than Random Forest with less than **1%** accuracy increase

## Using RBF kernel for SVM
-----

In [24]:
rbf_clf = svm.SVC(random_state=1)
rbf_clf.fit(x_train_norm,y_train)

rbfsvc_test_pred = rbf_clf.predict(x_test_norm)
rbfsvc_train_pred = rbf_clf.predict(x_train_norm)
train_acc_rbfsvc = sum(rbfsvc_train_pred == y_train)/x_train_norm.shape[0]
test_acc_rbfsvc = sum(rbfsvc_test_pred == y_test)/x_test_norm.shape[0]
print('RBF kernal SVC train accuracy:{0}'.format(train_acc_rbfsvc))
print('RBF kernal SVC test accuracy:{0}'.format(test_acc_rbfsvc))

RBF kernal SVC train accuracy:0.9930022817577403
RBF kernal SVC test accuracy:0.9924859357173768


The RBF Kernel SVM classifer is doing best among three models with test accuracy around **99.2%**